In [1]:
import spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
python -m spacy download en

In [2]:
nlp = spacy.load("en_core_web_lg")
nlp

In [3]:
doc = nlp("Donad Trump was President of USA")

In [4]:
doc

Donad Trump was President of USA

In [5]:
type(doc)

spacy.tokens.doc.Doc

In [6]:
doc.ents

(Donad Trump, USA)

In [7]:
doc.ents[0], type(doc.ents[0])

(Donad Trump, spacy.tokens.span.Span)

In [8]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [9]:
import json
with open('payment_terms_.json', 'rb') as f:
    data = json.load(f)

In [10]:
data['examples'][0]

{'id': 783,
 'text': "(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice.",
 'meta': {},
 'annotation_approver': None,
 'labels': [[41, 48, 'Payment Term']]}

In [11]:
data['examples'][0].keys()

dict_keys(['id', 'text', 'meta', 'annotation_approver', 'labels'])

In [12]:
data['examples'][0]['text']

"(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice."

In [13]:
data['examples'][67]['labels']

[[72, 92, 'Payment Term'], [188, 203, 'Payment Term']]

In [14]:
data['examples'][67]['meta']

{}

In [15]:
training_data = []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['text']
  temp_dict['entities'] = []
  temp_dict['entities'] = example['labels']
  training_data.append(temp_dict)
  
print(training_data[0])

{'text': "(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice.", 'entities': [[41, 48, 'Payment Term']]}


In [16]:
training_data[0]['text']

"(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice."

In [17]:
training_data[67]['entities']

[[72, 92, 'Payment Term'], [188, 203, 'Payment Term']]

In [18]:
training_data[0]['text'][41:48]

'30 days'

In [19]:
training_data[67]['text'][188:203]

'sixty (60) days'

In [56]:
def Enquiry(lis1):
    if len(lis1) == 0:
        return 0
    else:
        return 1

In [77]:
train_valid_data =[]
for i in range(len(training_data)):
    if Enquiry(training_data[i]['entities']):
        train_valid_data.append(training_data[i])


In [69]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()

In [70]:
from spacy.util import filter_spans

for training_example  in tqdm(train_valid_data): 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy") 

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 778.85it/s]


In [ ]:
# https://spacy.io/usage/training#quickstart                     copy configuration from this website

In [71]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [72]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy 

[i] Saving to output directory: .

[2023-04-10 15:34:04,829] [INFO] Set up nlp object from config
[2023-04-10 15:34:04,829] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-04-10 15:34:04,844] [INFO] Created vocabulary
[2023-04-10 15:34:06,281] [INFO] Added vectors: en_core_web_lg
[2023-04-10 15:34:07,547] [INFO] Finished initializing nlp object
[2023-04-10 15:34:08,109] [INFO] Initialized pipeline components: ['tok2vec', 'ner']



[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.50    0.00    0.00    0.00    0.00
 16     200         39.93    701.53  100.00  100.00  100.00    1.00
 37     400          0.32      2.02  100.00  100.00  100.00    1.00
 63     600          0.00      0.00  100.00  100.00  100.00    1.00
 95     800          0.00      0.00  100.00  100.00  100.00    1.00
135    1000          0.00      0.00  100.00  100.00  100.00    1.00
184    1200          0.00      0.00  100.00  100.00  100.00    1.00
246    1400          0.00      0.00  100.00  100.00  100.00    1.00
313    1600          0.00      0.00  100.00  100.00  100.00

In [75]:
nlp_ner = spacy.load("model-best")

In [76]:
doc = nlp_ner("i will deposit payment within 6 days ")

colors = {"PAYMENT TERM": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [78]:
doc = nlp_ner("(c)mrbwill pay undisputed amounts within 30 days after receiving Supplier's invoice.")

colors = {"PAYMENT TERM": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [79]:
doc = nlp_ner("i have given you 30 thousand rupees, i am expecting it back with 10 weeks ")

colors = {"PAYMENT TERM": "#F67DE3"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)